In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd 
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
#Ce projet porte sur la classificaion de créatures.
#Il existe trois types de créatures : Ghost (Fantôme), Goblin (Goblin) et Ghoul (Vampire). Chaque créature a des caractéristiques.
#Les caractéristiques sont : la longueur des os, le pourcentage de chair putréfié, la longueur des cheveux, le pourcentage d'âme et la couleur.
#Le but de ce projet est de classier des créatures avec des caractéristiques données. 

In [ ]:
#J'importe mes données qui me serviront de données d'entrainement et de données de test.
data=pd.read_csv('/kaggle/input/ghouls-goblins-and-ghosts-boo/train.csv.zip')
print(data)

In [ ]:
#Nous avons 371 lignes pour 7 colonnes
data.shape

In [ ]:
#Je visualise mes données, pour la sortie nous avons trois types de créatures. 0 pour Goblin, 1 pour fantômes et 2 pour Ghoul
data.head()

In [ ]:
#Je regarde mes créatures par rapport à leurs types et à leurs couleurs 
#On remarque que quelque soit le type de créatures, la couleur ne permet pas de les différencier selon les autres paramètres.
data.groupby(['type','color']).mean()

In [ ]:
#On remarque que le paramètre "Rotting_Flesh" ne nous sera pas utile pour différencier les créatures
sns.pairplot(data,hue='type') 

In [ ]:
#Je retire les colonnes "id", "color" et "rotting_flesh" qui ne seront pas utiles pour l'analyse des données. 
data=data.drop(['id','color', 'rotting_flesh'],axis=1)

In [ ]:
#Je crée maintenant mes données d'entrée et de sortie, en indiquant les colonnes d'entrée et de sortie. 
x=data.iloc[:,0:3]
y=data.iloc[:,3]
#Je crée mes données d'entrainement et de test. Les données de test vont représenter 20% des données de base. 
x_train, x_test, y_train, y_test = train_test_split(x,y,random_state=0,test_size=0.2)


In [ ]:
#J'utilise la méthode des KNeighbors pour classifier mes données. 
#Avec mes données d'entrainement, j'entraine mon algorithme. 
#Pour classer une créature, j'utilise, ici, les 5 créatures dont les paramètres se rapprochent le plus de ce dernier pour la classer
classifier = KNeighborsClassifier(n_neighbors=5, p=3)
classifier.fit(x_train,y_train)
#Une fois mes données entrainées, je prédis mes données de sortie par rapport à mon jeu de test
y_pred=classifier.predict(x_test)
#Ensuite, je compare mes données de sortie de mon jeu de test et les données prédies par ma méthode des KNeighbors.
#Pour cela, j'utilise la matrice de confusion 
cm = confusion_matrix (y_test, y_pred)
print(cm)
#Je calcule ensuite le score 
print(accuracy_score(y_test,y_pred))
#Nous avons un score de 0.746
#Ce score est plutôt satisfaisant en regardant les graphiques de nos données qui ne montraient pas des caractéristiques spécifiques pour chaque fantôme. 

In [ ]:
#Nous allons utiliser une seconde méthode celle de Naïve Bayésienne
classifier2 = GaussianNB()
#J'entraine mon modèle avec mes données d'entrainement comme précédemment
classifier2.fit(x_train, y_train)
#Je prédis avec mon deuxième modèle des données de sortie 
y_pred2 = classifier2.predict(x_test)
#J'affiche la matrice de confusion
cm2 = confusion_matrix(y_test, y_pred2)
print(cm2)
#Je vérifie la précision de mon deuxième modèle
score = accuracy_score(y_pred2, y_test)
print(score)
#Nous obtenon sun score de 0.706, ce qui est moins bien qu'avec notre premier modèle.

In [ ]:
#La troisème méthode est celle des arbres de décisions qui va déterminer des règles pour classer nos valeurs à partir des données d'apprentisage.
Classifier3 = DecisionTreeClassifier(max_depth=None)
#J'entraine mon troisième modèle
Classifier3.fit(x_train, y_train)
#Je calcule le score de mon modèle par rapport à mes données de test
Classifier3.score(x_test, y_test)
#Le score est de 0.626, ce qui est encore en dessous des deux modèles précédentes. 

In [ ]:
#J'importe mes données de test pour compléter le fichier qui sera soumis sur Kaggle 
data2=pd.read_csv('/kaggle/input/ghouls-goblins-and-ghosts-boo/test.csv.zip')
data2=data2.drop(['color', 'rotting_flesh'],axis=1)
x_test2=data2.iloc[:,1:4]
Y_id=data2.iloc[:,0]
print(x_test2)

In [ ]:
#J'utilise donc la méthode des KNeighbors pour classer mes données. Avec ce modèle, plus de 70% de mes données pourront être déterminées de manière assez fiable.
y_predit=classifier.predict(x_test2)
print(y_predit)

In [ ]:
#Je crée mon tableau avec l'id et la prédiction des créatures
submission=pd.DataFrame({'id':Y_id, 'type': y_predit})
print(submission)

In [ ]:
#Je transforme ensuite mon tableau en fichier csv pour l'envoyer sur Kaggle
submission.to_csv("submission.csv",index=False)